# 🧬 Gene Embedding Extraction Tutorial (scGPT)

**Last updated:** November 2025  

This tutorial demonstrates how to extract **gene embeddings** from **foundation models** — namely, **scGPT** (Transformer-based) and **CellFM** (MindSpore-based) — and store them directly inside an **AnnData** object (`adata.varm`).  

---

## Overview

**Workflow summary:**

1. Load the single-cell data (`AnnData`)
2. Load pretrained scGPT and/or CellFM models
3. Extract gene embeddings from each model
4. Store embeddings in `adata.varm`
5. Inspect and export results (optional)

> ⚠️ The pipeline below only depends on PyTorch + scGPT.

---

## 0. Requirements

Make sure you have the following packages installed:

```bash
pip install scanpy anndata pandas numpy scgpt

## 1. Parameter Setup

In [1]:
# your anndata file paths
adata_path = "/data/project/bin000815/scRNA/PriorGraph/Data/covid.h5ad"
out_h5ad  = "/data/project/bin000815/scRNA/PriorGraph/Data/covid_fm.h5ad"

# adata_path = "/path/to/adata.h5ad"
# out_h5ad  = "/path/to/adata_fm.h5ad"

# scGPT model directory
scgpt_model_dir = "/data/project/bin000815/scRNA/PriorGraph/scDECA/pretrained/scGPT"
# scgpt_model_dir = "/path/to/scDECA/scDECA/pretrained/scGPT"

human_flag = True  # Set to False for mouse data

## 2. Load the Helper Function

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from Embedding_extracter import add_gene_embeddings_to_adata

## 3. Load AnnData

In [3]:
import scanpy as sc
adata = sc.read_h5ad(adata_path)
adata

AnnData object with n_obs × n_vars = 26947 × 28696
    obs: 'donor_id', 'Peak_Respiratory_Support_WHO_Score', 'Bloody_Swab', 'Percent_Mitochondrial', 'SARSCoV2_PCR_Status', 'SARSCoV2_PCR_Status_and_WHO_Score', 'Cohort_Disease_WHO_Score', 'biosample_id', 'SingleCell_SARSCoV2_RNA_Status', 'SARSCoV2_Unspliced_TRS_Total_Corrected', 'SARSCoV2_Spliced_TRS_Total_Corrected', 'SARSCoV2_NegativeStrand_Total_Corrected', 'SARSCoV2_PositiveStrand_Total_Corrected', 'SARSCoV2_Total_Corrected', 'species', 'species__ontology_label', 'sex', 'disease', 'disease__ontology_label', 'organ', 'organ__ontology_label', 'library_preparation_protocol', 'library_preparation_protocol__ontology_label', 'age', 'Coarse_Cell_Annotations', 'Detailed_Cell_Annotations', 'label', 'cell_type_annotation', 'patient'
    var: 'n_cells'

## 4. Extract and Attach Gene Embeddings (scGPT only)

In [4]:
adata = add_gene_embeddings_to_adata(
    adata,
    scgpt_model_dir=scgpt_model_dir
)    

==> Adding scGPT gene embeddings
   - Total genes: 28696

=== scGPT ===
[scGPT] Loading from /data/project/bin000815/scRNA/PriorGraph/scDECA/pretrained/scGPT
[scGPT] Partial load: Error(s) in loading state_dict for TransformerModel:
	Missing key(s) in state_dict: "transformer_encoder.layers.0.self_attn.in_proj_weight", "transformer_encoder.layers.0.self_attn.in_proj_bias", "transformer_encoder.layers.1.self_attn.in_proj_weight", "transformer_encoder.layers.1.self_attn.in_proj_bias", "transformer_encoder.layers.2.self_attn.in_proj_weight", "transformer_encoder.layers.2.self_attn.in_proj_bias", "transformer_encoder.layers.3.self_attn.in_proj_weight", "transformer_encoder.layers.3.self_attn.in_proj_bias", "transformer_encoder.layers.4.self_attn.in_proj_weight", "transformer_encoder.layers.4.self_attn.in_proj_bias", "transformer_encoder.layers.5.self_attn.in_proj_weight", "transformer_encoder.layers.5.self_attn.in_proj_bias", "transformer_encoder.layers.6.self_attn.in_proj_weight", "transf

## 5. Inspect Results


In [5]:
import numpy as np

print("varm keys:", list(adata.varm.keys()))

if "scGPT_gene_token" in adata.varm:
    arr = adata.varm["scGPT_gene_token"]
    print("\n[scGPT_gene_token] shape:", arr.shape)
    n_nan = int(np.isnan(arr).all(axis=1).sum())
    print(" - Genes with all-NaN values:", n_nan)
    print(" - Embedding dimension:", arr.shape[1])

if "scGPT_gene_token_info" in adata.uns:
    print("\nuns['scGPT_gene_token_info']:", adata.uns["scGPT_gene_token_info"])

varm keys: ['scGPT_gene_token']

[scGPT_gene_token] shape: (28696, 512)
 - Genes with all-NaN values: 7514
 - Embedding dimension: 512

uns['scGPT_gene_token_info']: {'embedding_dim': 512, 'total_genes': 28696, 'genes_with_embeddings': 21182, 'missing_genes': 7514}


## 6. Save Updated AnnData

In [ ]:
adata.write_h5ad(out_h5ad)
print("Saved:", out_h5ad)